#### 회귀 분석

In [12]:
"""
회귀 분석은 수치형 변수 간의 관계 또는 원인과 결과 관계를 예측하는데 사용
상관 계수 - 변수간의 강도와 방향을 측정하는 계수로 두 변수 간의 선형적인 관계가 
어느정도 강한지를 나타냄.

"""
import pandas as pd

data = {
    "키": [150, 160, 170, 175, 165],
    "몸무게": [42, 50, 70, 64, 56]
}
df = pd.DataFrame(data)
df

# 상관 계수 (피어슨) - method="pearson"(생략)
correlation = df.corr()
print(correlation)

print(df["키"].corr(df["몸무게"]))
print(df["몸무게"].corr(df["키"]))

            키       몸무게
키    1.000000  0.919509
몸무게  0.919509  1.000000
0.9195090879163765
0.9195090879163765


#### 선형 회귀 분석(단순선형, 다중 선형)

In [57]:
"""
# 단순 선형 회귀
# 독립변수 1개와 종속 변수 1개로 이루어짐
# 키가 몸무게에 미치는 영향을 분석해서 회귀 모델 구축
# statsmodels.formula.api
# '~' 독립변수 ~ 종속변수 
"""
import pandas as pd

data = {
    '키': [150, 160, 170, 175, 165, 155, 172, 168, 174, 158,
          162, 173, 156, 159, 167, 163, 171, 169, 176, 161],
    '몸무게': [42, 50, 70, 64, 56, 48, 68, 60, 65, 52,
            54, 67, 49, 51, 58, 55, 69, 61, 66, 53]
}
df = pd.DataFrame(data)
df

from statsmodels.formula.api import ols

# formula - "키 ~ 몸무게"
model = ols("키 ~ 몸무게", data=df).fit()
print(model.summary())
# help(ols)
# 결정 계수 - 모델이 그 데이터를 얼마나 잘 설명하느냐를 나타내는 통계값
# 1에 가까울 수록 잘 설명함(R2)
print("결정계수:", model.rsquared)

# 회귀 계수 - y=ax + b(a:기울기, b:y절편), y= 0.86x + 115
print("회귀계수: ", model.params)
print("기울기: ", model.params['몸무게'])
print("절편: ", model.params["Intercept"])
print("p-value: ", model.pvalues["몸무게"]) #지수표기법
print(f"p-value: {model.pvalues["몸무게"]:.10f}") #일반 표기법

# 평가 지표 - mse(평균제곱오차), mae(평균절대값오차), rmse(평균제곱근오차)
# 잔차 = 실제값-예측값
df['잔차'] = df['키'] - model.predict(df)
df
# 잔차 제곱합
print((df['잔차']**2).sum())

# MSE(평균 제곱 오차)
df['잔차'] = df['키'] - model.predict(df)
mse = (df['잔차']**2).mean()
print("MSE:", mse)

# 몸무게가 67일때 키를 예측
new_df = pd.DataFrame({"몸무게": [72]})
result = model.predict(new_df)
print(result)
# print(round(result[0], 1))

                            OLS Regression Results                            
Dep. Variable:                      키   R-squared:                       0.892
Model:                            OLS   Adj. R-squared:                  0.886
Method:                 Least Squares   F-statistic:                     148.0
Date:                Mon, 18 Nov 2024   Prob (F-statistic):           4.04e-10
Time:                        17:48:27   Log-Likelihood:                -45.761
No. Observations:                  20   AIC:                             95.52
Df Residuals:                      18   BIC:                             97.51
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    115.0676      4.158     27.671      0.0